* 예제 핵심 : 새로운 DICOM 파일을 생성하여 file meta information, attribute을 설정하고 저장한다. 
    * File meta information은 Dataset을 통해 생성하고, 그 외 attribute 들은 FileDataset을 통해 생성한다. 
    * liitle endian, implicit VR 에 대한 설정을 한다. 
    * <'FileDataset' class>의 save_as 를 통해 저장한다. 
    * 본 예제에는 Pixel Data에 대한 생성은 없음**
* os.remove : 파일 삭제 
* tempfile : 임시 파일을 생성하기 위한 모듈. 임시 파일에 맞게 이름을 적당한 랜덤으로 생성해준다. 확장자(suffix)를 지정할 수 있고, 그외 디렉토리 등도 원하면 지정할 수 있는 듯. (https://docs.python.org/3/library/tempfile.html)

In [3]:
# authors : Guillaume Lemaitre <g.lemaitre58@gmail.com>
# license : MIT

import os
import tempfile
import datetime

import pydicom
from pydicom.dataset import Dataset, FileDataset

# Create some temporary filenames
suffix = '.dcm'
filename_little_endian = tempfile.NamedTemporaryFile(suffix=suffix).name
print(filename_little_endian)
filename_big_endian = tempfile.NamedTemporaryFile(suffix=suffix).name

print("Setting file meta information...")
# Populate required values for file meta information
file_meta = Dataset()
file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'
file_meta.MediaStorageSOPInstanceUID = "1.2.3"
file_meta.ImplementationClassUID = "1.2.3.4"

print("Setting dataset values...")
# Create the FileDataset instance (initially no data elements, but file_meta
# supplied)
ds = FileDataset(filename_little_endian, {},
                 file_meta=file_meta, preamble=b"\0" * 128)

# Add the data elements -- not trying to set all required here. Check DICOM
# standard
ds.PatientName = "Test^Firstname"
ds.PatientID = "123456"

# Set the transfer syntax
ds.is_little_endian = True
ds.is_implicit_VR = True

# Set creation date/time
dt = datetime.datetime.now()
ds.ContentDate = dt.strftime('%Y%m%d')
timeStr = dt.strftime('%H%M%S.%f')  # long format with micro seconds
ds.ContentTime = timeStr

print("Writing test file", filename_little_endian)
ds.save_as(filename_little_endian)
print("File saved.")

# Write as a different transfer syntax XXX shouldn't need this but pydicom
# 0.9.5 bug not recognizing transfer syntax
ds.file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRBigEndian
ds.is_little_endian = False
ds.is_implicit_VR = False

print("Writing test file as Big Endian Explicit VR", filename_big_endian)
ds.save_as(filename_big_endian)

# reopen the data just for checking
for filename in (filename_little_endian, filename_big_endian):
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    print(ds)

    # remove the created file
    print('Remove file {} ...'.format(filename))
    os.remove(filename)

/var/folders/q0/054bp2592gbc54h12rv1gdq00000gn/T/tmpnxwm5zu3.dcm
Setting file meta information...
Setting dataset values...
Writing test file /var/folders/q0/054bp2592gbc54h12rv1gdq00000gn/T/tmpnxwm5zu3.dcm
File saved.
Writing test file as Big Endian Explicit VR /var/folders/q0/054bp2592gbc54h12rv1gdq00000gn/T/tmp28hd9d4j.dcm
Load file /var/folders/q0/054bp2592gbc54h12rv1gdq00000gn/T/tmpnxwm5zu3.dcm ...
(0008, 0023) Content Date                        DA: '20190311'
(0008, 0033) Content Time                        TM: '232144.066926'
(0010, 0010) Patient's Name                      PN: 'Test^Firstname'
(0010, 0020) Patient ID                          LO: '123456'
Remove file /var/folders/q0/054bp2592gbc54h12rv1gdq00000gn/T/tmpnxwm5zu3.dcm ...
Load file /var/folders/q0/054bp2592gbc54h12rv1gdq00000gn/T/tmp28hd9d4j.dcm ...
(0008, 0023) Content Date                        DA: '20190311'
(0008, 0033) Content Time                        TM: '232144.066926'
(0010, 0010) Patient's Name        